In [2]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import warnings
warnings.filterwarnings('ignore')

from lists import dyads, num_dyads, participants, num_participants

In [3]:
# Set sampling rate
sampling_rate = 2000
print("Sampling rate:", sampling_rate)

# Set conditions
condition = ["sit","gaze"]
print("Conditions:", condition)

# Show number of participants
print("Participants:", num_participants)

# Show number of dyads
print("Dyads:", num_dyads)

Sampling rate: 2000
Conditions: ['sit', 'gaze']
Participants: 8
Dyads: 4


In [4]:
# Calculate heart rates
heartrate_temp = []
dict_temp = {}
heartrates = np.zeros((len(condition), num_dyads, 2))

for i, con in enumerate(condition):
    ecg_raw_2000hz = np.load(f"data/ecg_raw/ecg_raw_{con}.npy")
    for j in range(num_participants):
        ecg = nk.ecg_clean(ecg_raw_2000hz[j], sampling_rate=sampling_rate, method="neurokit")
        r_peaks_temp, intervals_temp = nk.ecg_peaks(
            ecg, sampling_rate=sampling_rate, method="neurokit", correct_artifacts=True)
        heartrate_temp.append(sum(r_peaks_temp["ECG_R_Peaks"])/5) # 5 minutes
        dict_temp[i] = heartrate_temp
    heartrate_temp = []
    print(f"Average heart rates per person in condition {con.upper()}")
    # Save data as 3D array
    for k, idx in enumerate(list(range(num_participants))[::2]): # index for dyads
        heartrates[i][k][0] = dict_temp[i][idx]
        heartrates[i][k][1] = dict_temp[i][idx+1]
        print(f"Dyad {k}: Female: {dict_temp[i][idx]} Male: {dict_temp[i][idx+1]}")

# Build dataframe
print("\nAverage heart rates and SD over all participants")

heartrate_mean = pd.DataFrame(
    {"Female": [np.mean(heartrates[0, :, 0]), np.mean(heartrates[1, :, 0]), np.mean(heartrates[:, :, 0])],
     "Male": [np.mean(heartrates[0, :, 1]), np.mean(heartrates[1, :, 1]), np.mean(heartrates[:, :, 1])],
     "All": [np.mean(heartrates[0, :, :]), np.mean(heartrates[1, :, :]), np.mean(heartrates[:, :, :])]},
    index=["SIT", "GAZE", "All"])
heartrate_mean = heartrate_mean.round(2)
print(heartrate_mean.T)

heartrate_std = pd.DataFrame(
    {"Female": [np.std(heartrates[0, :, 0]), np.std(heartrates[1, :, 0]), np.std(heartrates[:, :, 0])],
     "Male": [np.std(heartrates[0, :, 1]), np.std(heartrates[1, :, 1]), np.std(heartrates[:, :, 1])],
     "All": [np.std(heartrates[0, :, :]), np.std(heartrates[1, :, :]), np.std(heartrates[:, :, :])]},
    index=["SIT", "GAZE", "All"])
heartrate_std = heartrate_std.round(2)
print(heartrate_std.T)

Average heart rates per person in condition SIT
Dyad 0: Female: 72.4 Male: 60.2
Dyad 1: Female: 70.0 Male: 69.6
Dyad 2: Female: 92.6 Male: 76.2
Dyad 3: Female: 82.0 Male: 77.2
Average heart rates per person in condition GAZE
Dyad 0: Female: 70.8 Male: 60.8
Dyad 1: Female: 73.4 Male: 66.8
Dyad 2: Female: 91.6 Male: 78.4
Dyad 3: Female: 74.8 Male: 72.6

Average heart rates and SD over all participants
          SIT   GAZE    All
Female  79.25  77.65  78.45
Male    70.80  69.65  70.23
All     75.03  73.65  74.34
         SIT  GAZE   All
Female  8.92  8.18  8.60
Male    6.78  6.55  6.69
All     8.98  8.42  8.73


In [5]:
print(f"delta mean SIT: {np.mean(heartrates[0, :, 0]) - np.mean(heartrates[0, :, 1]):.2f}")
print(f"delta mean GAZE: {np.mean(heartrates[1, :, 0]) - np.mean(heartrates[1, :, 1]):.2f}")
print(f"delta std SIT: {np.std(heartrates[0, :, 0]) - np.std(heartrates[0, :, 1]):.2f}")
print(f"delta std GAZE: {np.std(heartrates[1, :, 0]) - np.std(heartrates[1, :, 1]):.2f}")

delta mean SIT: 8.45
delta mean GAZE: 8.00
delta std SIT: 2.14
delta std GAZE: 1.63
